# DAML 06 - sklearn Introduction

Michal Grochmal <michal.grochmal@city.ac.uk>

The `sklearn` module in Python tries to *standardize the way machine learning
techniques are performed*.  Instead of technique specific setup and manual application
of the resulting model, `sklearn` proposes a standard class-based API.
Every ML technique in `sklean` uses the following steps to produce a single model.

1.  Every ML technique (model type) is a class that you can import
2.  You feed the *hyperparameters* to the constructor, a new instance of a model
    is bound to the hyperparameters passed in
3.  You call `.fit()` on the input data, which has a standardized form, see below
4.  For models that predict data you use `.predict()` to appl the model
5.  For models that transform the data you use `.transform()`, some models have
    a `.fit_transform()` which will fit and transform the same data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-talk')
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

## Input and Output

Injecting data and reading the outputs of a model can often be troublesome,
figuring out in which format a model wants its parameters is no easy task.
Developers of `sklearn` tried to sort the misery of different libraries
requiring parameters in different formats, and created a standard way in which
all `sklearn` models receive the input.

Ignore the code below, this is just to draw the picture below.

In [ ]:
fig, ax = plt.subplots(figsize=(9, 6))
ax.axis('off')
ax.axis('equal')
ax.vlines(range(4), ymin=0, ymax=6, lw=1)
ax.hlines(range(7), xmin=0, xmax=3, lw=1)
ax.vlines(range(4, 6), ymin=0, ymax=6, lw=1)
ax.hlines(range(7), xmin=4, xmax=5, lw=1)
ax.text(1.5, 6.5, "input / data / X", size=14, ha='center')
ax.text(4.6, 6.5, "target / y", size=14, ha='center')
ax.text(0, -0.3, r'features $\longrightarrow$')
samples = r'$\longleftarrow$ samples'
ax.text(0, 6, samples, rotation=90, va='top', ha='right')
ax.text(5.1, 6, samples, rotation=90, va='top', ha='left')
ax.set_ylim(-1, 7);

At first one may argue that this will only work for two dimensions,
but if you remember the ideas of *group by* and *pivot* we can see that
we can represent several dimensions in a two dimensional table.

## Hyperparameters

What the hell are those *hyperparameters*?
These are simply free parameters that interfere in the inner workings of the model.
By *free* we mean that the model works independently of what we set these parameters to.
Yet, the values we give influence how well the model performs against a specific dataset.

Therefore yes, the entire practice of machine learning is about finding the right model
and the right model hyperparameters.  This could not be harder though, the combination of
several free parameters goes into millions or thousands of millions for some models.

As a simple example let's implement $k$ nearest neighbors, a model that has a single
hyperparameter ($k$).

In [ ]:
class MyKNN(object):
    def __init__(self, k=1):
        self.k_ = k

    def fit(self, X, y):
        self.X = X
        self.y = y

    def predict(self, new_X):
        sq_dist_dim = (new_X[:, np.newaxis, :] - self.X[np.newaxis, :, :])**2
        sq_dist = sq_dist_dim.sum(axis=-1)
        self.nearest_ = np.argpartition(sq_dist, self.k_, axis=1)
        new_y = self.y[self.nearest_[:, :self.k_]]
        new_y = np.apply_along_axis(lambda x: np.bincount(x, minlength=2), axis=1, arr=new_y)
        return new_y.argmax(axis=1)

The actual work in our algorithm happen during `argpartition`,
which sorts the points according to their distances from each point stored in the class.
The we just sum together the classes of all close points (`bincount`),
and output the most common class (`argmax`).

Our implementation is imperfect,
in case of a tie the algorithm prefers the class with the lowest index.
But this is irrelevant if we use our algorithm as a binary classifier and with an odd $k$.
A full implementation will need to deal with the ties.

Nevertheless, we can try out our algorithm by classifying some data.

In [ ]:
random_state = 27
rng = np.random.RandomState(random_state)
X, y = make_blobs(30, 2, centers=2, cluster_std=0.2, center_box=(0, 1), random_state=random_state)
new_X = rng.rand(6, 2)
knn_left = MyKNN(k=1)
knn_left.fit(X, y)
new_yl = knn_left.predict(new_X)
knn_right = MyKNN(k=5)
knn_right.fit(X, y)
new_yr = knn_right.predict(new_X)
fig, ax = plt.subplots(2, 2, figsize=(16, 10))
fig.subplots_adjust(hspace=0, wspace=0)
ax[0, 0].set_title('k =  1')
ax[0, 0].scatter(X[:, 0], X[:, 1], s=50, c=y, cmap='viridis')
ax[0, 0].scatter(new_X[:, 0], new_X[:, 1], s=200)
ax[1, 0].scatter(X[:, 0], X[:, 1], s=50, c=y, cmap='viridis')
ax[1, 0].scatter(new_X[:, 0], new_X[:, 1], s=200, c=new_yl, cmap='viridis')
ax[0, 1].set_title('k =  5')
ax[0, 1].scatter(X[:, 0], X[:, 1], s=50, c=y, cmap='viridis')
ax[0, 1].scatter(new_X[:, 0], new_X[:, 1], s=200)
ax[1, 1].scatter(X[:, 0], X[:, 1], s=50, c=y, cmap='viridis')
ax[1, 1].scatter(new_X[:, 0], new_X[:, 1], s=200, c=new_yr, cmap='viridis')
for axi in ax.flat:
    axi.axis('equal')
    axi.set_xticks([], [])
    axi.set_yticks([], [])
new_yl, new_yr

The algorithm works but the difference in the classification when we change $k$.
We need a way to tell what is a good value for $k$.
Since we have several points for which we know the label already we can use
them to check whether we get the right label.  This should be easy.

How **not** to do it:

In [ ]:
knn = MyKNN(k=1)
knn.fit(X, y)
new_y = knn.predict(X)
error = (y - new_y)**2
error.mean()

A perfect model!  Which means $k = 5$ must be wrong, right?

In [ ]:
knn = MyKNN(k=5)
knn.fit(X, y)
new_y = knn.predict(X)
error = (y - new_y)**2
error.mean()

There is something fishy about this.  Can you tell?

Let's get some help from `sklearn`

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=random_state, test_size=0.3)
accuracy = []
f1_scores = []
k_vals = list(range(1, 16))
for k in k_vals:
    knn = MyKNN(k=k)
    knn.fit(train_X, train_y)
    new_y = knn.predict(test_X)
    accuracy.append(accuracy_score(test_y, new_y))
    f1_scores.append(f1_score(test_y, new_y))

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(k_vals, accuracy, color='crimson')
ax.plot(k_vals, f1_scores, color='steelblue');

That looks promising let's see how this goes on a bigger set.
And we should try it a couple of times
to see if we see any patterns.

In [ ]:
def select_k(ax):
    X, y = make_blobs(512, 2, centers=2, cluster_std=0.2, center_box=(0, 1))
    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3)
    accuracy = []
    f1_scores = []
    k_vals = list(range(1, 256))
    for k in k_vals:
        knn = MyKNN(k=k)
        knn.fit(train_X, train_y)
        new_y = knn.predict(test_X)
        accuracy.append(accuracy_score(test_y, new_y))
        f1_scores.append(f1_score(test_y, new_y))
    ax.plot(k_vals, accuracy, color='crimson')
    ax.plot(k_vals, f1_scores, color='steelblue')
    ax.set(xlabel='$k$', ylabel='score')


fig, ax = plt.subplots(3, 1, figsize=(16, 14), sharex=True)
fig.subplots_adjust(hspace=0)
for axi in ax.flat:
    select_k(axi)

There are [many, many ways to calculate the "correctness" of what a model did spit out][metrics].
And several of these are better for specific problems than others.  As a rule of thumb,
a "good enough" first try measure is the F1 score, which performs well enough for most problems.
It is defined in terms of *precision* and *recall*:

#### precision

$$P = \frac{true\:positives}{true\:positives + false\:positives}$$

#### recall

$$R = \frac{true\:positives}{true\:positives + false\:negatives}$$

#### F1 score

$$F_1 = \frac{1}{\frac{1}{P} + \frac{1}{R}}=2 \frac{P \cdot R}{P + R}$$

In this model we have only a single free parameter,
and it is integer valued which makes it easy to select by hand.
Some models have dozens of real valued parameters making the search
for an optimal parameter much harder.
We will see that when we look at these models.

[metrics]: http://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics

## References

-   [Evaluation of retrieved sets/classes - IR intro][1]

[1]: https://nlp.stanford.edu/IR-book/html/htmledition/evaluation-of-unranked-retrieval-sets-1.html